In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Social_Media_Analysis/final exam') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['TaipeiSansTCBeta-Regular.ttf',
 'data',
 'dict',
 'net_func.py',
 'ex1_undirected_graph.html',
 'final_exam_1.ipynb',
 'lda_zh.html',
 'final_exam_2.ipynb',
 'final_exam_3.ipynb']

In [2]:
!pip install jieba

!pip install sentence-transformers
!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
!pip install langchain_community
!pip install langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.2 MB/s eta 0:00:00


In [5]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# 資料前處理

In [8]:
df = pd.read_csv("./data/emotion_samples.csv")
df.head(3)

,text,label
0,I cant sleep im too worried about ashton and i...,negative
1,Having to be at school at eight tomorrow is go...,negative
2,Three hours of sleep would be a lot worse if i...,negative


In [9]:
#去除文章內容為空值的筆數
df.dropna(subset = ['text'], axis=0, how='any', inplace=True)
df=df.rename(columns={'text': 'sentence'})
#新增['sentence']欄位，用'。'取代'\n\n'，並移除'\n'
df['sentence'] = df['sentence'].str.replace(r'\n\n','。', regex=True)
df['sentence'] = df['sentence'].str.replace(r'\n','', regex=True)

#移除內文中的網址
df['sentence'] = df['sentence'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

df['sentence'] = df['sentence'].str.split("[,，。！!？?]{1,}")

sent_df = df.explode('sentence').reset_index(drop=True)
# 把長度小於1的sentence刪掉
sent_df = sent_df[sent_df["sentence"].str.len() > 1]


# HuggingFace Token

In [13]:
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


### HuggingFacePipeline

In [11]:
quant_config = BitsAndBytesConfig(load_in_4bit=True)

print(quant_config.is_quantizable())
print(quant_config.quantization_method())

True
fp4


In [14]:
model_id = "google/gemma-1.1-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGINGFACEHUB_API_TOKEN)
# device_map="auto" 使用 GPU 加速
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quant_config, token=HUGGINGFACEHUB_API_TOKEN)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [24]:
print(sent_df.iloc[0:3])

                                            sentence     label
0  I cant sleep im too worried about ashton and i...  negative
1  Having to be at school at eight tomorrow is go...  negative
2  Three hours of sleep would be a lot worse if i...  negative


# Few-shot + self-consistency

In [ ]:
fewshot_template = """<start_of_turn>user
You are a sentiment analysis expert. You will help analyze the sentiment of the given text.

Here are some examples:
```
Question: I am so happy today because I got a promotion!
Answer：positive

Question: I feel really sad and lonely.
Answer：negative
```

Now please analyze the following text:
Question: {question}<end_of_turn>
<start_of_turn>model
Answer:"""

fewshot_prompt = PromptTemplate.from_template(fewshot_template)

fewshot_chain = (
    {"question": RunnablePassthrough()}
    | fewshot_prompt
    | llm
)
fewshot_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？")

In [ ]:
for i in range(1, 6):
  print(f"\n\n========== Run {i} ==========\n\n")
  print(fewshot_chain.invoke("當我六歲的時候，我妹妹的年齡只有我的一半。 現在我70歲了，我妹妹幾歲了？"))

In [39]:
class Food(BaseModel):
    情緒: str = Field(..., description="判斷整個句子總體的情緒。", enum=["positive", "neural", "negative"])

food_parser = JsonOutputParser(pydantic_object=Food)

# 透過 get_format_instructions() 可以看到如何告訴 LLM 生成我們想要的格式的一些指令
food_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u500b\\u53e5\\u5b50\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["positive", "neural", "negative"], "type": "string"}}, "required": ["\\u60c5\\u7dd2"]}\n```'

In [40]:
# Set up a parser + inject instructions into the prompt template.
food_parser = JsonOutputParser(pydantic_object=Food)\

food_template = """<start_of_turn>user
你是一位文字情緒分析師，你將會看網路上的評論，請抓取出評論中：
此評論的情緒，positive或negative或neutral


以下為一些範例：
```
句子: I am so happy today because I got a promotion!
抓取結果：{{
    "情緒":"positive"
}}

句子: I feel really sad and lonely.
抓取結果：{{
    "情緒":"negative"
}}

句子: The weather today is just okay, neither good nor bad.
抓取結果：{{
    "情緒":"neutral"
}}



{format_instructions}
請使用英文回答。
除了 JSON 內容外，不要回答任何其他內容

現在請幫我抓取以下句子的資訊：
句子：{query}
抓取結果：<end_of_turn>
<start_of_turn>model
"""

food_prompt = PromptTemplate(
    template=food_template,
    # template="Answer the user query.\n{format_instructions}\nDon't output any other explanation\n{query}\n",
    input_variables=["query"],
    # 將 format_instructions 也輸入到 prompt templete 中
    partial_variables={"format_instructions": food_parser.get_format_instructions()},
)

chain = food_prompt | llm
chain.get_graph().print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
      +----------------+       
      | PromptTemplate |       
      +----------------+       
              *                
              *                
              *                
   +---------------------+     
   | HuggingFacePipeline |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| HuggingFacePipelineOutput |  
+---------------------------+  


In [41]:
# 從 json 字串中抓取答案
def extract_answer_from_string(string_):
    string_ = string_.replace("\n", "")
    string_ = string_.replace("json", "")
    string_ = string_.replace("    ", "")
    string_ = string_.replace("```", "")
    try:
        # 可以成功轉換
        dict_result = json.loads(string_)
    except:
        print(string_)
        return string_
    return dict_result

# 從 dict 抓取情緒
def get_sentiment(row):
    sentiment = None
    if isinstance(row, dict):
        if ("sentiment" in row.keys()):
            sentiment = row['sentiment']
    return sentiment

In [44]:
chain.invoke({"query": "The weather today is just okay, neither good nor bad."})

'<start_of_turn>user\n你是一位文字情緒分析師，你將會看網路上的評論，請抓取出評論中：\n此評論的情緒，positive或negative或neutral\n\n\n以下為一些範例：\n```\n句子: I am so happy today because I got a promotion!\n抓取結果：{\n    "情緒":"positive"\n}\n\n句子: I feel really sad and lonely.\n抓取結果：{\n    "情緒":"negative"\n}\n\n句子: The weather today is just okay, neither good nor bad.\n抓取結果：{\n    "情緒":"neutral"\n}\n\n\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u500b\\u53e5\\u5b50\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["positive

In [43]:
llm_result = extract_answer_from_string(chain.invoke({"query": "The weather today is just okay, neither good nor bad."}))
print(llm_result)
# get_sentiment(llm_result)

<start_of_turn>user你是一位文字情緒分析師，你將會看網路上的評論，請抓取出評論中：此評論的情緒，positive或negative或neutral以下為一些範例：句子: I am so happy today because I got a promotion!抓取結果：{"情緒":"positive"}句子: I feel really sad and lonely.抓取結果：{"情緒":"negative"}句子: The weather today is just okay, neither good nor bad.抓取結果：{"情緒":"neutral"}The output should be formatted as a JSON instance that conforms to the JSON schema below.As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.Here is the output schema:{"properties": {"\u60c5\u7dd2": {"title": "\u60c5\u7dd2", "description": "\u5224\u65b7\u6574\u500b\u53e5\u5b50\u7e3d\u9ad4\u7684\u60c5\u7dd2\u3002", "enum": ["positive", "neural", "negative"], "type": "string"}}, "required": ["\u60c5\u7dd2"]}請使用英文回答。除了 JSON 內容外，不要

In [ ]:
sampled_df['LLM'] = sampled_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
sampled_df

In [ ]:
sampled_df['sentiment'] = sampled_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)
sampled_df.head()

In [ ]:
sampled_df['sentiment'].value_counts()